<div class='status' style='background-color: #183a4b; color: white; padding-top: 4px; padding-bottom: 4px; padding-left: 20px; padding-right: 20px; border-radius: 10px; font-family: Arial, sans-serif; font-size: 26px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.5);'><b>Fake GP Reviews - FAKE Reviews SCORER</div>

## Loading Libraries

In [6]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import datetime
import os
from tqdm import tqdm

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
warnings.filterwarnings("ignore")

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

<div class='status' style='background-color: #f5c244; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Loading</b> dataset</div>

In [56]:
fake_data = pd.read_csv("../ai_medreview/data/fake_gp_reviews_03_processed.csv")
fake_data.head(2)

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels,emotion_free_text,label,abs_sentiment
0,Reception staff were kind but disorganized som...,Patient Respect,158,neutral,0.500156,Reception Staff Interaction,admiration,1,0.500156
1,"The GP surgery staff were extremely helpful, a...",Patient Respect,353,positive,0.716657,Reception Staff Interaction,disappointment,1,0.716657


In [45]:
fake_data.columns

Index(['free_text', 'generation_category', 'free_text_len',
       'sentiment_free_text', 'sentiment_score_free_text', 'feedback_labels',
       'emotion_free_text', 'rating'],
      dtype='object')

In [46]:
fake_data['label'] = 1

In [47]:
fake_data['abs_sentiment'] = fake_data.apply(
    lambda row: -row['sentiment_score_free_text'] if row['sentiment_free_text'] == 'negative' else row['sentiment_score_free_text'],
    axis=1
)

In [48]:
fake_data.head(20)

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels,emotion_free_text,rating,label,abs_sentiment
0,The GP practice is run by a friendly woman who...,Patient Respect,878,positive,0.873504,Reception Staff Interaction,disappointment,4,1,0.873504
1,Staff were very friendly but slow. \n\nThe GP ...,Patient Respect,88,positive,0.441180,Reception Staff Interaction,disappointment,4,1,0.441180
2,The GP surgery staff were kind and attentive d...,Patient Respect,385,positive,0.590509,Reception Staff Interaction,disappointment,4,1,0.590509
3,The staff were friendly and efficient making m...,Patient Respect,266,positive,0.922695,Reception Staff Interaction,disappointment,4,1,0.922695
4,The staff were friendly and accommodating.\nTh...,Patient Respect,116,positive,0.920581,Test Results,admiration,3,1,0.920581
5,The GP surgery staff were very helpful and und...,Patient Respect,598,positive,0.916409,Reception Staff Interaction,disappointment,3,1,0.916409
6,The GP practice takes a caring approach toward...,Patient Respect,267,positive,0.889275,Patient Respect,admiration,4,1,0.889275
7,The GP surgery staff were very helpful and exp...,Patient Respect,236,positive,0.594936,Reception Staff Interaction,disappointment,4,1,0.594936
8,The reception staff were friendly and welcomin...,Patient Respect,200,positive,0.441538,Reception Staff Interaction,disappointment,2,1,0.441538
9,The staff were friendly and helpful during my ...,Patient Respect,662,negative,0.475945,Reception Staff Interaction,disappointment,4,1,-0.475945


# Scorer

In [35]:
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [51]:
from jan883_codebase.llm.ollama import ask_ollama

In [ ]:
score_list = []

for review in tqdm(fake_data['free_text'], desc=f"Scoring: ", total=fake_data.shape[0]):
    prompt = f"""Your task is to score the GP Surgery on the question:
“Overall, how was your experience of our service?”

You have already written your review:

{review}

Scoring Criteria (0 - 5 scale):
	•	5 = Very good
	•	4 = Good
	•	3 = Neither good nor poor
	•	2 = Poor
	•	1 = Very poor
	•	0 = Don’t know

🔹 How to score:
	•	If your review is highly positive, confidently score 5 or 4.
	•	If your review is mixed or neutral, use 3 appropriately.
	•	If your review is negative, score 2 or 1, depending on the severity of the criticism.
	•	If the review expresses uncertainty or lacks clarity, consider 0 (Don’t know).

🛑 Output only the score as an INTEGER (0-5). Do NOT include any explanations, text, or comments.
🎯 Be decisive with your score—let it reflect the tone of the review.

⚠️ Remember: This is a FAKE GP Surgery review. Your score should reflect the sentiment in the review realistically."""
    
    output = ask_ollama(
        prompt,
        system_prompt='You are a patient at a GP Surgery who is an expert at writing fake reviews.',
        model='nezahatkorkmaz/deepseek-v3:latest',
        temp=1.3,
    )

    score_list.append(output)


Scoring:   4%|██                                                 | 78/1935 [01:34<31:25,  1.02s/it]

In [ ]:
# Final save, to ensure we capture any remaining reviews 
fake_data['rating'] = score_list

In [ ]:
fake_data.to_csv('../ai_medreview/data/fake_gp_reviews_03_processed.csv', index=False)

In [39]:
df1.head()

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels,emotion_free_text,label,abs_sentiment
0,The staff were very helpful and kind.\n\nI rec...,Patient Respect,246,positive,0.959466,Reception Staff Interaction,disappointment,1,0.959466
1,The staff were friendly but forgot my appointm...,Patient Respect,933,negative,0.878586,Communication Effectiveness,disappointment,1,-0.878586
2,The staff were kind but the wait was long. The...,Patient Respect,53,positive,0.910060,Reception Staff Interaction,admiration,1,0.910060
3,The GP was very understanding of my concerns b...,Patient Respect,86,positive,0.602609,Environment and Ambiance,approval,1,0.602609
4,I found the GP surgery to be thoroughly disorg...,Patient Respect,105,negative,0.866430,Patient Respect,disappointment,1,-0.866430


In [40]:
df2.head()

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels,emotion_free_text,rating
0,The GP practice is run by a friendly woman who...,Patient Respect,878,positive,0.873504,Reception Staff Interaction,disappointment,4
1,Staff were very friendly but slow. \n\nThe GP ...,Patient Respect,88,positive,0.441180,Reception Staff Interaction,disappointment,4
2,The GP surgery staff were kind and attentive d...,Patient Respect,385,positive,0.590509,Reception Staff Interaction,disappointment,4
3,The staff were friendly and efficient making m...,Patient Respect,266,positive,0.922695,Reception Staff Interaction,disappointment,4
4,The staff were friendly and accommodating.\nTh...,Patient Respect,116,positive,0.920581,Test Results,admiration,3


In [41]:
df3.head()

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels,emotion_free_text,label,abs_sentiment
0,Reception staff were kind but disorganized som...,Patient Respect,158,neutral,0.500156,Reception Staff Interaction,admiration,1,0.500156
1,"The GP surgery staff were extremely helpful, a...",Patient Respect,353,positive,0.716657,Reception Staff Interaction,disappointment,1,0.716657
2,The reception staff were friendly and welcomin...,Patient Respect,85,positive,0.588108,Reception Staff Interaction,admiration,1,0.588108
3,Reception staff were unfriendly and slow but g...,Patient Respect,53,negative,0.477082,Reception Staff Interaction,annoyance,1,-0.477082
4,The staff at this GP surgery were extremely he...,Patient Respect,178,positive,0.939486,Staff Professionalism,approval,1,0.939486
